In [15]:
#google_search_scrapper_tool
import sys
import subprocess

# Ensure required modules are installed
def install_packages():
    packages = ["requests", "beautifulsoup4", "pandas", "google-search-results"]
    for package in packages:
        try:
            __import__(package)
        except ImportError:
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])

install_packages()

from serpapi import GoogleSearch
import pandas as pd

def google_search_scraper(query, num_results=10):
    """
    Fetches Google search results using SerpAPI.
    """
    api_key = "bcc39452a15e256ee9a03213aede7be60ffc66bb4e044be17296963493fef2ab"  # Replace with your actual API key
    params = {
        "q": query,
        "num": num_results,
        "api_key": api_key
    }

    search = GoogleSearch(params)
    results = search.get_dict()
    search_results = []

    if "organic_results" in results:
        for result in results["organic_results"]:
            title = result.get("title", "No Title")
            link = result.get("link", "No Link")
            snippet = result.get("snippet", "No Description")
            search_results.append({"Title": title, "Link": link, "Description": snippet})
    else:
        print("No results found.")

    return search_results

def save_to_csv(results, filename="google_search_results.csv"):
    """
    Saves the search results to a CSV file.
    """
    df = pd.DataFrame(results)
    df.to_csv(filename, index=False)
    print(f"Results saved to {filename}")

if __name__ == "__main__":
    query = input("Enter search query: ")
    results = google_search_scraper(query)
    if results:
        save_to_csv(results)
    else:
        print("No results found.")

Enter search query: Latest Tesla News
Results saved to google_search_results.csv


In [16]:
#youyube_video_scrapper_tool
import sys
import subprocess

def install_packages():
    packages = ["requests", "beautifulsoup4", "pandas", "google-search-results"]
    for package in packages:
        try:
            __import__(package)
        except ImportError:
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])

install_packages()

import requests
import pandas as pd

def youtube_video_scraper(query, max_results=10):
    """
    Fetches YouTube video results using SerpAPI.
    """
    api_key = "bcc39452a15e256ee9a03213aede7be60ffc66bb4e044be17296963493fef2ab"  # Replace with your actual SerpAPI key
    url = "https://serpapi.com/search"
    params = {
        "engine": "youtube",
        "search_query": query,
        "api_key": api_key,
        "num": max_results
    }

    response = requests.get(url, params=params)
    data = response.json()

    video_results = []
    for item in data.get("video_results", []):
        title = item.get("title", "No Title")
        video_url = item.get("link", "No URL")
        description = item.get("snippet") or item.get("rich_snippet") or item.get("description", "No Description")
        channel_info = item.get("channel", {})
        channel_name = channel_info.get("name", "Unknown Channel")
        views = item.get("views", "Unknown Views")

        video_results.append({
            "Title": title,
            "URL": video_url,
            "Description": description,
            "Channel Name": channel_name,
            "Views": views
        })

    return video_results

def save_to_csv(results, filename="youtube_video_results.csv"):
    """
    Saves the YouTube video results to a CSV file.
    """
    df = pd.DataFrame(results)
    df.to_csv(filename, index=False)
    print(f"Results saved to {filename}")

if __name__ == "__main__":
    query = input("Enter YouTube search query: ")
    results = youtube_video_scraper(query)
    if results:
        save_to_csv(results)
    else:
        print("No results found.")

Enter YouTube search query: Breaking World News Today
Results saved to youtube_video_results.csv


In [18]:
#summarized_data_scrapper_tool
import sys
import subprocess

def install_packages():
    packages = ["requests", "pandas"]
    for package in packages:
        try:
            __import__(package)
        except ImportError:
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])

install_packages()

import requests
import pandas as pd

def summarize_with_serpapi(query, api_key):
    """
    Uses SerpAPI to fetch summarized results for a given query.
    """
    url = "https://serpapi.com/search"
    params = {
        "engine": "google",
        "q": query,
        "api_key": api_key
    }

    response = requests.get(url, params=params)
    data = response.json()

    # Extract AI-generated answers or snippets
    summary = data.get("answer_box", {}).get("snippet") or \
              data.get("organic_results", [{}])[0].get("snippet", "No Summary Available")

    return summary

def process_csv(file_path, api_key):
    """
    Reads a CSV file, queries SerpAPI for summaries, and outputs key insights.
    """
    df = pd.read_csv(file_path)
    if "Title" not in df.columns:
        print("No 'Title' column found.")
        return

    df["Summary"] = df["Title"].apply(lambda x: summarize_with_serpapi(str(x), api_key) if pd.notna(x) else "No Summary")
    output_file = "summarized_data.csv"
    df.to_csv(output_file, index=False)
    print(f"Summarized results saved to {output_file}")

if __name__ == "__main__":
    csv_file = input("Enter CSV file path: ")
    api_key = input("Enter SerpAPI key: ")
    process_csv(csv_file, api_key)


Enter CSV file path: /content/google_search_results.csv
Enter SerpAPI key: bcc39452a15e256ee9a03213aede7be60ffc66bb4e044be17296963493fef2ab
Summarized results saved to summarized_data.csv
